In [ ]:
import torch
import cv2
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}, {torch.cuda.get_device_name(torch.cuda.current_device())}')

Using device: cuda, NVIDIA L4


In [ ]:
!pip install ultralytics

In [ ]:
# alterado para 10 epochs para uma execução mais rapida para efeitos de demonstração

from ultralytics import YOLO

model = YOLO("yolov9c.pt")
model.info()

results = model.train(data="/content/dataset/data.yaml", epochs=100, imgsz=416, batch=64, workers=16)

100%|██████████| 49.4M/49.4M [00:00<00:00, 370MB/s]


YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs
Ultralytics YOLOv8.2.45 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/content/dataset/data.yaml, epochs=10, time=None, patience=100, batch=64, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=16, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


AMP: checks passed ✅


train: Scanning /content/dataset/train/labels.cache... 5115 images, 2 backgrounds, 0 corrupt: 100%|██████████| 5115/5115 [00:00<?, ?it/s]

...

      10/10      11.2G      1.023      0.594      1.244         74        320: 100%|██████████| 80/80 [00:41<00:00,  1.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.66it/s]

                   all        225        316      0.915      0.756      0.828      0.575



10 epochs completed in 0.161 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv9c summary (fused): 384 layers, 25320019 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


                   all        225        316      0.915      0.756      0.828      0.575
Speed: 0.0ms preprocess, 2.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


In [ ]:
model.save("/content/trained_yolov9c.pt")
results = model("input.mp4")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/631) /content/input.mp4: 224x320 1 LicensePlate, 22.8ms
video 1/1 (frame 2/631) /content/input.mp4: 224x320 1 LicensePlate, 16.7ms
video 1/1 (frame 3/631) /content/input.mp4: 224x320 1 LicensePlate, 21.3ms
video 1/1 (frame 4/631) /content/input.mp4: 224x320 2 LicensePlates, 16.5ms
video 1/1 (frame 5/631) /content/input.mp4: 224x320 1 LicensePlate, 17.6ms
video 1/1 (frame 6/631) /content/input.mp4: 224x320 2 LicensePlates, 16.5ms
vide

In [ ]:
def predict_and_detect(model, img, conf=0.5, rectangle_thickness=2, text_thickness=2):
    results = model.predict(source=img, conf=conf)  # Use the YOLO model to predict
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), rectangle_thickness)
            text = f"License Plate: {box.conf[0].item():.2f}"
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), text_thickness)
    return img

In [ ]:
def create_video_writer(video_cap, output_filename):
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    return writer

In [ ]:
def process_video(input_video_path, output_filename, model, conf=0.5):
    output_dir = "/content/outputs"
    os.makedirs(output_dir, exist_ok=True)

    output_video_path = os.path.join(output_dir, output_filename)

    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Unable to open file.")
        return

    writer = create_video_writer(cap, output_video_path)

    frame_count = 0
    while True:
        success, img = cap.read()
        if not success:
            print("Error: Unable to read frame or video has ended.")
            break

        result_img = predict_and_detect(model, img, conf=conf)

        writer.write(result_img)
        cv2.waitKey(1)

        frame_count += 1
        print(f"Frame {frame_count} processed.")

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"The video has been processed successfully and saved to {output_video_path}.")

In [ ]:
# first run with 0.5 confidence

input_video_path = "input.mp4"
output_video_path = "output-0.5confidence.mp4"
process_video(input_video_path, output_video_path, model, conf=0.5)


0: 224x320 1 LicensePlate, 19.2ms
Speed: 1.9ms preprocess, 19.2ms inference, 1.6ms postprocess per image at shape (1, 3, 224, 320)
Frame 1 processed.

0: 224x320 1 LicensePlate, 18.5ms
Speed: 1.1ms preprocess, 18.5ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 2 processed.

0: 224x320 1 LicensePlate, 19.1ms
Speed: 1.0ms preprocess, 19.1ms inference, 1.6ms postprocess per image at shape (1, 3, 224, 320)
Frame 3 processed.

0: 224x320 1 LicensePlate, 17.7ms
Speed: 1.0ms preprocess, 17.7ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 4 processed.

0: 224x320 1 LicensePlate, 19.5ms
Speed: 1.3ms preprocess, 19.5ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 5 processed.

0: 224x320 1 LicensePlate, 21.6ms
Speed: 1.0ms preprocess, 21.6ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 6 processed.

0: 224x320 1 LicensePlate, 16.7ms
Speed: 1.0ms preprocess, 16.7ms inference, 1.4ms postproces

In [ ]:
# second run with 0.1 confidence

input_video_path = "input.mp4"
output_video_path = "output-0.1confidence.mp4"
process_video(input_video_path, output_video_path, model, conf=0.1)


0: 224x320 2 LicensePlates, 18.6ms
Speed: 1.4ms preprocess, 18.6ms inference, 1.6ms postprocess per image at shape (1, 3, 224, 320)
Frame 1 processed.

0: 224x320 1 LicensePlate, 17.0ms
Speed: 0.9ms preprocess, 17.0ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 2 processed.

0: 224x320 2 LicensePlates, 17.9ms
Speed: 1.0ms preprocess, 17.9ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 3 processed.

0: 224x320 2 LicensePlates, 17.1ms
Speed: 1.1ms preprocess, 17.1ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 4 processed.

0: 224x320 1 LicensePlate, 17.7ms
Speed: 1.2ms preprocess, 17.7ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 5 processed.

0: 224x320 2 LicensePlates, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 6 processed.

0: 224x320 3 LicensePlates, 16.8ms
Speed: 1.1ms preprocess, 16.8ms inference, 1.4ms postp

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install easyocr

In [ ]:
import easyocr
import numpy as np

def detect_color(roi):

  hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
  colors = {
        "Red": ([0, 120, 70], [10, 255, 255]),
        "Green": ([36, 25, 25], [70, 255, 255]),
        "Blue": ([94, 80, 2], [126, 255, 255]),
        "Yellow": ([15, 150, 150], [35, 255, 255]),
        "Black": ([0, 0, 0], [180, 255, 30]),
        "White": ([0, 0, 200], [180, 20, 255]),
        "Gray": ([0, 0, 40], [180, 20, 200])
    }
  max_pixels = 0
  dominant_color = "Unknown"

  for color, (lower, upper) in colors.items():
      mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
      num_pixels = cv2.countNonZero(mask)

      if num_pixels > max_pixels:
          max_pixels = num_pixels
          dominant_color = color

  return dominant_color

In [ ]:
# update predict_and_detect to use EasyOCR and color detection

reader = easyocr.Reader(['en'])

def predict_and_detect(model, img, conf=0.5, rectangle_thickness=2, text_thickness=2):
    results = model.predict(source=img, conf=conf)  # Use the YOLO model to predict
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            plate_img = img[y1:y2, x1:x2]

            expand_y = int((y2 - y1) * 4)
            expand_x = int((x2 - x1) * 3)
            y1_expanded = max(y1 - expand_y // 2, 0)
            y2_expanded = min(y2 + expand_y // 2, img.shape[0])
            x1_expanded = max(x1 - expand_x // 2, 0)
            x2_expanded = min(x2 + expand_x // 2, img.shape[1])

            car_roi = img[y1_expanded:y2_expanded, x1_expanded:x2_expanded]

            ocr_result = reader.readtext(plate_img)
            license_plate_text = "Unknown"
            if ocr_result:
                license_plate_text = ocr_result[0][-2]

            car_color = detect_color(car_roi)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), rectangle_thickness)
            text = f"{license_plate_text}, {car_color}"
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), text_thickness)
    return img

In [ ]:
# run again with easyocr and color detection

input_video_path = "input.mp4"
output_video_path = "output-final.mp4"
process_video(input_video_path, output_video_path, model, conf=0.5)


0: 224x320 1 LicensePlate, 17.5ms
Speed: 1.2ms preprocess, 17.5ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 1 processed.

0: 224x320 1 LicensePlate, 17.9ms
Speed: 1.3ms preprocess, 17.9ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 2 processed.

0: 224x320 1 LicensePlate, 19.7ms
Speed: 1.1ms preprocess, 19.7ms inference, 2.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 3 processed.

0: 224x320 1 LicensePlate, 17.9ms
Speed: 1.3ms preprocess, 17.9ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 4 processed.

0: 224x320 1 LicensePlate, 17.3ms
Speed: 1.1ms preprocess, 17.3ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 5 processed.

0: 224x320 1 LicensePlate, 17.0ms
Speed: 0.9ms preprocess, 17.0ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 6 processed.

0: 224x320 1 LicensePlate, 17.0ms
Speed: 0.9ms preprocess, 17.0ms inference, 1.4ms postproces

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Frame 385 processed.

0: 224x320 1 LicensePlate, 17.3ms
Speed: 1.3ms preprocess, 17.3ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 320)
Frame 386 processed.

0: 224x320 1 LicensePlate, 16.8ms
Speed: 1.1ms preprocess, 16.8ms inference, 1.3ms postprocess per image at shape (1, 3, 224, 320)
Frame 387 processed.

0: 224x320 1 LicensePlate, 16.9ms
Speed: 0.9ms preprocess, 16.9ms inference, 1.3ms postprocess per image at shape (1, 3, 224, 320)
Frame 388 processed.

0: 224x320 1 LicensePlate, 16.9ms
Speed: 1.0ms preprocess, 16.9ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 389 processed.

0: 224x320 1 LicensePlate, 23.7ms
Speed: 1.1ms preprocess, 23.7ms inference, 1.8ms postprocess per image at shape (1, 3, 224, 320)
Frame 390 processed.

0: 224x320 1 LicensePlate, 17.9ms
Speed: 1.3ms preprocess, 17.9ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 320)
Frame 391 processed.

0: 224x320 1 LicensePlate, 18.3ms
Speed: 1.2ms preprocess, 1